In [1]:
import geopandas as gpd
import pandas as pd
# Create a folium heatmap of the assessed difference for properties
import folium
from folium.plugins import HeatMap
# That didn't load enough data, the query limit is 65000, so let's try a different approach
# Base URL for the Parcel Points feature layer in GeoJSON format
base_url = "https://geodata.baltimorecity.gov/egis/rest/services/CityView/Realproperty_OB/FeatureServer/0/query"
max_records = 1000  # Maximum number of records per query


def fetch_data(base_url, max_records):
    all_data = gpd.GeoDataFrame()
    offset = 0

    while True:
        query_params = {
            "where": "VACIND%20%3D%20'Y'",
            "outFields": "PIN,VACIND",
            "outSR": "4326",
            "f": "geojson",
            "resultOffset": offset,
            "resultRecordCount": max_records,
        }
        query_url = f"{base_url}?{'&'.join([f'{k}={v}' for k, v in query_params.items()])}"

        data_chunk = gpd.read_file(query_url)

        if data_chunk.empty:
            break

        all_data = pd.concat([all_data, data_chunk], ignore_index=True)
        offset += max_records

    return all_data


gdf = fetch_data(base_url, max_records)
# Convert to projected CRS
gdf = gdf.to_crs(epsg=3857)
# Convert to points
gdf["geometry"] = gdf["geometry"].centroid
# Convert back to lat/lon
gdf = gdf.to_crs(epsg=4326)
print(f"Number of records: {len(gdf)}")
print(f"Number of unique PINs: {len(gdf['PIN'].unique())}")

gdf.head()

Number of records: 13554
Number of unique PINs: 13544


,PIN,VACIND,geometry
0,0001003,Y,POINT (-76.65104 39.30943)
1,0001004,Y,POINT (-76.65099 39.30943)
2,0001007,Y,POINT (-76.65084 39.30943)
3,0001008,Y,POINT (-76.65080 39.30943)
4,0001013,Y,POINT (-76.65055 39.30944)


In [2]:


# Create a base map
m = folium.Map(
    location=[gdf.geometry.y.mean(), gdf.geometry.x.mean()],
    tiles="cartodbpositron",
    zoom_start=11,
)

# Heatmap of the assessed difference
heat_data = [
    [row["geometry"].y, row["geometry"].x] for index, row in gdf.iterrows()
]

In [5]:
HeatMap(heat_data, radius=5, blur=7, min_opacity=0.3, max_zoom=13, gradient={0.2: "blue", 0.4: "lime", 0.6: "yellow", 1: "red"}).add_to(m)

m